In [1]:
import sys

import numpy as np
import pandas as pd
import s3fs
import seaborn as sns

sys.path.append("/Users/fgu/dev/projects/entropy")
import entropy.data.aggregators as agg
import entropy.data.cleaners as cl
import entropy.data.make_data as md
import entropy.data.selectors as sl
import entropy.data.validators as vl
import entropy.helpers.aws as ha
import entropy.helpers.data as hd
import entropy.helpers.helpers as hh

pd.set_option("display.max_rows", 120)
pd.set_option("display.max_columns", 120)
pd.set_option("max_colwidth", None)
%load_ext autoreload
%autoreload 2

fs = s3fs.S3FileSystem(profile="3di")

In [ ]:
df = hd.read_txn_data("XX7")
hd.inspect(df)

In [100]:
pre_reg = (df.date < df.user_registration_date).astype(int)
pre_reg.groupby(df.user_id).mean().describe()

count    3103.000000
mean        0.419298
std         0.344665
min         0.000000
25%         0.069655
50%         0.366071
75%         0.719323
max         1.000000
dtype: float64